In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("satrapankti/amazon-beauty-product-recommendation")

print("Path to dataset files:", path)

c:\Users\Abdul Haseeb\Desktop\Recommender system\Recommender-System\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\Abdul Haseeb\.cache\kagglehub\datasets\satrapankti\amazon-beauty-product-recommendation\versions\1


In [7]:
#reading the csv file
import pandas as pd
data = pd.read_csv(r'C:\Users\Abdul Haseeb\Desktop\Recommender system\Recommender-System\Amazon_Beauty_Recommendation.csv')
print(data.head())

           UserId   ProductId       ProductType  Rating   Timestamp  \
0  A3NHUQ33CFH3VM  B00LLPT4HI  Eyeliner & Kajal       5  1405814400   
1  A1TIRNQ7O4REOH  B00LLPT4HI  Eyeliner & Kajal       4  1405987200   
2  A2Y36BR4YSY9F7  B00LLPT4HI  Eyeliner & Kajal       5  1405728000   
3  A23H6FAOLEMAKC  B00LLPT4HI  Eyeliner & Kajal       5  1405814400   
4  A3CHYZGF3OO6WD  B00LLPT4HI  Eyeliner & Kajal       5  1405641600   

                                                 URL  
0  https://www.amazon.in/Maybelline-Colossal-Kaja...  
1  https://www.amazon.in/Maybelline-Colossal-Kaja...  
2  https://www.amazon.in/Maybelline-Colossal-Kaja...  
3  https://www.amazon.in/Maybelline-Colossal-Kaja...  
4  https://www.amazon.in/Maybelline-Colossal-Kaja...  


In [8]:
#data preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Display basic info about the dataset
print("Dataset Info:")
print(data.info())

# Handle missing values (if any)
data = data.dropna()

# Remove duplicates (if any)
data = data.drop_duplicates()

# Encode UserId and ProductId into numeric format
user_encoder = LabelEncoder()
product_encoder = LabelEncoder()

data['UserId'] = user_encoder.fit_transform(data['UserId'])
data['ProductId'] = product_encoder.fit_transform(data['ProductId'])

# Feature and Target
features = data[['UserId', 'ProductId', 'Rating', 'Timestamp']]
target = data['Rating']

# Normalizing Timestamp (improves model performance)
features = features.copy()
features['Timestamp'] = (features['Timestamp'] - features['Timestamp'].min()) / (features['Timestamp'].max() - features['Timestamp'].min())

prepared_data_path = 'prepared_data.csv'
features.to_csv(prepared_data_path, index=False)
print(f"Prepared data saved to {prepared_data_path}")

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1348246 entries, 0 to 1348245
Data columns (total 6 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   UserId       1348246 non-null  object
 1   ProductId    1348246 non-null  object
 2   ProductType  1348246 non-null  object
 3   Rating       1348246 non-null  int64 
 4   Timestamp    1348246 non-null  int64 
 5   URL          1348246 non-null  object
dtypes: int64(2), object(4)
memory usage: 61.7+ MB
None
Prepared data saved to prepared_data.csv


In [9]:
# Split the data into training and testing sets
X = features[['UserId', 'ProductId', 'Timestamp']]
y = features['Rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the splits
print(f"Training Features Shape: {X_train.shape}")
print(f"Testing Features Shape: {X_test.shape}")
print(f"Training Target Shape: {y_train.shape}")
print(f"Testing Target Shape: {y_test.shape}")

# Save the splits for later use
X_train.to_csv('X_train.csv', index=False)
X_test.to_csv('X_test.csv', index=False)
y_train.to_csv('y_train.csv', index=False)
y_test.to_csv('y_test.csv', index=False)

print("Training and testing splits saved as CSV files.")

Training Features Shape: (1078596, 3)
Testing Features Shape: (269650, 3)
Training Target Shape: (1078596,)
Testing Target Shape: (269650,)
Training and testing splits saved as CSV files.


In [10]:
#implemeting model
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate, Dropout

# Define the number of unique users and products
num_users = X_train['UserId'].nunique()
num_products = X_train['ProductId'].nunique()

# Embedding size - hyperparameter
embedding_size = 50

# Input layers
user_input = Input(shape=(1,), name='User_Input')
product_input = Input(shape=(1,), name='Product_Input')

# Embedding layers
user_embedding = Embedding(input_dim=num_users, output_dim=embedding_size, name='User_Embedding')(user_input)
product_embedding = Embedding(input_dim=num_products, output_dim=embedding_size, name='Product_Embedding')(product_input)

# Flatten the embeddings
user_vec = Flatten(name='Flatten_User')(user_embedding)
product_vec = Flatten(name='Flatten_Product')(product_embedding)

# Concatenate user and product vectors
concat = Concatenate(name='Concatenate')([user_vec, product_vec])

# Add dense layers (fully connected layers)
dense_1 = Dense(128, activation='relu', name='Dense_Layer_1')(concat)
dropout_1 = Dropout(0.3, name='Dropout_1')(dense_1)  # Dropout to prevent overfitting
dense_2 = Dense(64, activation='relu', name='Dense_Layer_2')(dropout_1)

# Output layer (rating prediction)
output = Dense(1, activation='linear', name='Output')(dense_2)

# Define the model
model = Model(inputs=[user_input, product_input], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

# Model summary
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ User_Input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Product_Input       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ User_Embedding      │ (None, 1, 50)     │ 37,112,550 │ User_Input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Product_Embedding   │ (None, 1, 50)     │  1,191,900 │ Product_Input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Flatten_User        │ (None, 50)        │          0 │ User_Embedding[0… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Flatten_Product     │ (None, 50)        │          0 │ Product_Embeddin… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Concatenate         │ (None, 100)       │          0 │ Flatten_User[0][… │
│ (Concatenate)       │                   │            │ Flatten_Product[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dense_Layer_1       │ (None, 128)       │     12,928 │ Concatenate[0][0] │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dropout_1 (Dropout) │ (None, 128)       │          0 │ Dense_Layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dense_Layer_2       │ (None, 64)        │      8,256 │ Dropout_1[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Output (Dense)      │ (None, 1)         │         65 │ Dense_Layer_2[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 38,325,699 (146.20 MB)

 Trainable params: 38,325,699 (146.20 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define input data for training
train_user_input = X_train['UserId'].values
train_product_input = X_train['ProductId'].values
train_ratings = y_train.values

val_user_input = X_test['UserId'].values
val_product_input = X_test['ProductId'].values
val_ratings = y_test.values

# Define early stopping and model checkpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(
    'best_model.keras', save_best_only=True, monitor='val_loss', mode='min'
)

# Train the model
history = model.fit(
    [train_user_input, train_product_input],
    train_ratings,
    validation_data=([val_user_input, val_product_input], val_ratings),
    epochs=10,  # Adjust based on need
    batch_size=256,
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)

# Save training history for visualization
import pickle
with open('training_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)

print("Model training completed and the best model saved as 'best_model.h5'.")


Epoch 1/10
4214/4214 ━━━━━━━━━━━━━━━━━━━━ 1276s 303ms/step - loss: 1.8993 - mean_absolute_error: 1.0778 - val_loss: 1.5337 - val_mean_absolute_error: 0.9647
Epoch 2/10
4214/4214 ━━━━━━━━━━━━━━━━━━━━ 1133s 269ms/step - loss: 1.0426 - mean_absolute_error: 0.7358 - val_loss: 1.7584 - val_mean_absolute_error: 0.9974
Epoch 3/10
4214/4214 ━━━━━━━━━━━━━━━━━━━━ 1130s 268ms/step - loss: 0.7408 - mean_absolute_error: 0.5624 - val_loss: 1.9321 - val_mean_absolute_error: 1.0595
Epoch 4/10
4214/4214 ━━━━━━━━━━━━━━━━━━━━ 1133s 269ms/step - loss: 0.5409 - mean_absolute_error: 0.4515 - val_loss: 1.9819 - val_mean_absolute_error: 1.0481
Model training completed and the best model saved as 'best_model.h5'.


In [17]:
# Load the best model saved during training
best_model = tf.keras.models.load_model('best_model.keras')

# Evaluate the model on the test set
test_loss, test_mae = best_model.evaluate([val_user_input, val_product_input], val_ratings, verbose=1)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Mean Absolute Error: {test_mae:.4f}")

8427/8427 ━━━━━━━━━━━━━━━━━━━━ 36s 4ms/step - loss: 1.5453 - mean_absolute_error: 0.9678
Test Loss: 1.5337
Test Mean Absolute Error: 0.9647


In [18]:
#unseen data making and testing
import numpy as np

# Randomly sample a subset from the test set as unseen data
unseen_indices = np.random.choice(X_test.index, size=1000, replace=False)  # Adjust size as needed
unseen_sample = X_test.loc[unseen_indices]
unseen_ratings = y_test.loc[unseen_indices]  # Corresponding ground truth ratings

# Prepare input for testing
unseen_user_input = unseen_sample['UserId'].values
unseen_product_input = unseen_sample['ProductId'].values

predicted_ratings = best_model.predict([unseen_user_input, unseen_product_input])

# Map predictions back to ProductType and URL using the original dataset
unseen_sample['Predicted_Rating'] = predicted_ratings

# Merge the unseen sample with the original dataset to get ProductType and URL
unseen_sample_with_details = unseen_sample.merge(
    data[['ProductId', 'ProductType', 'URL']],
    on='ProductId',
    how='left'
)

# Display the first few rows with predicted ratings, ProductType, and URL
print(unseen_sample_with_details[['ProductId', 'Predicted_Rating', 'ProductType', 'URL']].head(10))

print("Model input shapes:")
for input_shape in best_model.input_shape:
    print(input_shape)

unseen_user_input = tf.convert_to_tensor(unseen_user_input, dtype=tf.int32)
unseen_product_input = tf.convert_to_tensor(unseen_product_input, dtype=tf.int32)
unseen_ratings = tf.convert_to_tensor(unseen_ratings, dtype=tf.float32)

# Evaluate performance on the unseen sample
unseen_loss, unseen_mae = best_model.evaluate(
    [unseen_user_input, unseen_product_input], unseen_ratings, verbose=1
)

print(f"Simulated Unseen Data Loss: {unseen_loss:.4f}")
print(f"Simulated Unseen Data Mean Absolute Error: {unseen_mae:.4f}")

# Predict ratings for the unseen sample
predicted_ratings = best_model.predict([unseen_user_input, unseen_product_input])

print("Predicted Ratings (first 10):")
print(predicted_ratings[:10])

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step
   ProductId  Predicted_Rating ProductType  \
0       6680            4.2175  Shower Gel   
1       6680            4.2175  Shower Gel   
2       6680            4.2175  Shower Gel   
3       6680            4.2175  Shower Gel   
4       6680            4.2175  Shower Gel   
5       6680            4.2175  Shower Gel   
6       6680            4.2175  Shower Gel   
7       6680            4.2175  Shower Gel   
8       6680            4.2175  Shower Gel   
9       6680            4.2175  Shower Gel   

                                                 URL  
0  https://www.amazon.in/Cleansense-Gentle-Pamper...  
1  https://www.amazon.in/Cleansense-Gentle-Pamper...  
2  https://www.amazon.in/Cleansense-Gentle-Pamper...  
3  https://www.amazon.in/Cleansense-Gentle-Pamper...  
4  https://www.amazon.in/Cleansense-Gentle-Pamper...  
5  https://www.amazon.in/Cleansense-Gentle-Pamper...  
6  https://www.amazon.in/Cleansense-Gentle-Pamper...  
7  https://ww

In [20]:
#predictiing for a specific user
import numpy as np
import pandas as pd
import tensorflow as tf

# Get unique user IDs and product IDs from the dataset
unique_users = X_test['UserId'].unique()
unique_products = X_test['ProductId'].unique()

# Example: Generate recommendations for a specific user
target_user_id = unique_users[5]

# Debugging: Print the expected input shapes of the model
print("Model input shapes:")
for input_shape in best_model.input_shape:
    print(input_shape)

# Prepare inputs for predictions
target_user_input = tf.convert_to_tensor(np.array([target_user_id] * len(unique_products)).reshape(-1, 1), dtype=tf.int32)
all_product_input = tf.convert_to_tensor(unique_products.reshape(-1, 1), dtype=tf.int32)

# Debugging: Verify shapes of inputs
print(f"Shape of target_user_input: {target_user_input.shape}")
print(f"Shape of all_product_input: {all_product_input.shape}")

# Perform predictions
predicted_ratings = best_model.call([target_user_input, all_product_input])

# Convert predictions to a NumPy array
predicted_ratings = predicted_ratings.numpy().flatten()

# Create a DataFrame to display the top recommendations
recommendations = pd.DataFrame({
    'ProductId': unique_products,
    'Predicted_Rating': predicted_ratings
}).sort_values(by='Predicted_Rating', ascending=False)

# Merge with product details (e.g., ProductType and URL)
recommendations = recommendations.merge(
    data[['ProductId', 'ProductType', 'URL']].drop_duplicates(),
    on='ProductId',
    how='left'
)

# Display the top 10 recommendations for the target user
print("Top 10 Recommendations for User ID:", target_user_id)
print(recommendations[['ProductId', 'Predicted_Rating', 'ProductType', 'URL']].head(10))

Model input shapes:
(None, 1)
(None, 1)
Shape of target_user_input: (23684, 1)
Shape of all_product_input: (23684, 1)
Top 10 Recommendations for User ID: 654719
   ProductId  Predicted_Rating          ProductType  \
0      20431          4.982078          Conditioner   
1      23619          4.968677           Bath Salts   
2       8526          4.964384           Shower Gel   
3       8287          4.955808           Shower Gel   
4      23128          4.955686             Lipstick   
5       7092          4.951395           Shower Gel   
6      21688          4.951287  Cream & Moisturizer   
7        436          4.950228          Nail Polish   
8       2262          4.948620          Nail Polish   
9       8234          4.943204           Shower Gel   

                                                 URL  
0  https://www.amazon.in/Schwarzkopf-Professional...  
1  https://www.amazon.in/gp/slredirect/picassoRed...  
2  https://www.amazon.in/mCaffeine-Exfoliating-En...  
3  https://ww